In [0]:
#!pip install keras==1.2.2

In [0]:
#!pip install tensorflow==0.12.1
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Sem 6/video captioning/testings/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/gdrive/My Drive/Sem 6/video captioning/testings/'
/content


In [0]:
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from tqdm import tqdm
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
import nltk

Using TensorFlow backend.


In [0]:
token = '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Text/Flickr8k.token.txt'

In [0]:
captions = open(token, 'r').read().strip().split('\n')

In [0]:
d = {}
for i, row in enumerate(captions):
    row = row.split('\t')
    row[0] = row[0][:len(row[0])-2]
    if row[0] in d:
        d[row[0]].append(row[1])
    else:
        d[row[0]] = [row[1]]

In [0]:
images = '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Data/'

In [0]:
# Contains all the images
img = glob.glob(images+'*.jpg')
img[:5]               

['/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Data/2665461736_595c87f0a3.jpg',
 '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Data/3662871327_b128d25f04.jpg',
 '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Data/976392326_082dafc3c5.jpg',
 '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Data/3215117062_6e07a86352.jpg',
 '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Data/3286045254_696c6b15bd.jpg']

In [0]:
train_images_file = '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Text/Flickr_8k.trainImages.txt'
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

In [0]:
def split_data(l):
    temp = []
    for i in img:
        if i[len(images):] in l:
            temp.append(i)
    return temp

In [0]:
train_img = split_data(train_images)
len(train_img)

6000

In [0]:
val_images_file = '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Text/Flickr_8k.devImages.txt'
val_images = set(open(val_images_file, 'r').read().strip().split('\n'))

In [0]:
val_img = split_data(val_images)
len(val_img)

1000

In [0]:
test_images_file = '/content/gdrive/My Drive/Sem 6/video_captioning/testings/Flickr8K_Text/Flickr_8k.testImages.txt'
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

In [0]:
test_img = split_data(test_images)
len(test_img)

1000

In [0]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [0]:
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)
    return x

In [0]:
model = InceptionV3(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.
96116736/96112376 [==============================] - 1s 0us/step


In [0]:
train_d = {}
for i in train_img:
    if i[len(images):] in d:
        train_d[i] = d[i[len(images):]]

In [0]:
val_d = {}
for i in val_img:
    if i[len(images):] in d:
        val_d[i] = d[i[len(images):]]

In [0]:
test_d = {}
for i in test_img:
    if i[len(images):] in d:
        test_d[i] = d[i[len(images):]]

In [0]:
caps = []
for key, val in train_d.items():
    for i in val:
        caps.append('<start> ' + i + ' <end>')

In [0]:
words = [i.split() for i in caps]

In [0]:
unique = []
for i in words:
    unique.extend(i)

In [0]:
unique = list(set(unique))

In [0]:
unique = pickle.load(open('unique.p', 'rb'))

In [0]:
len(unique)

8256